In [1]:
import torch
import math
from Project2_tools import *
import matplotlib.pyplot as plt
torch.set_grad_enabled(False)
import Project2_tools

In [2]:
train_input, train_target = generate_set(1000)
test_input, test_target = generate_set(1000)

In [3]:
print(train_input)

tensor([[0.1305, 0.9627],
        [0.7021, 0.0680],
        [0.4448, 0.5816],
        ...,
        [0.4578, 0.4227],
        [0.8616, 0.2766],
        [0.5360, 0.1267]])


In [4]:
print(train_target)

tensor([-1, -1,  1,  1,  1,  1, -1,  1, -1, -1, -1, -1, -1,  1,  1, -1, -1,  1,
        -1,  1,  1, -1, -1,  1,  1,  1, -1,  1, -1, -1,  1, -1,  1, -1,  1,  1,
        -1, -1, -1, -1,  1, -1, -1,  1, -1,  1,  1, -1,  1,  1, -1,  1, -1,  1,
         1, -1,  1,  1,  1, -1,  1, -1, -1,  1, -1,  1, -1,  1,  1,  1,  1,  1,
         1,  1,  1,  1, -1,  1,  1, -1,  1, -1, -1,  1,  1, -1, -1, -1,  1,  1,
         1,  1,  1, -1, -1, -1, -1, -1, -1,  1,  1,  1, -1,  1,  1, -1,  1,  1,
         1,  1, -1, -1, -1, -1, -1,  1, -1,  1,  1,  1, -1, -1,  1, -1, -1, -1,
        -1,  1, -1,  1,  1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1, -1,  1, -1,
         1, -1, -1, -1, -1, -1,  1,  1,  1, -1, -1, -1,  1,  1,  1, -1,  1,  1,
         1, -1, -1, -1,  1,  1,  1, -1, -1,  1, -1,  1, -1, -1,  1,  1, -1,  1,
         1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1,  1, -1,  1,  1,  1,
         1, -1, -1,  1, -1,  1, -1,  1, -1, -1, -1,  1, -1, -1,  1, -1,  1,  1,
        -1, -1,  1, -1,  1, -1,  1, -1, 

In [5]:
#better results with normalosation
mean = train_input.mean()
std = train_input.std()
train_input = train_input-mean
train_input = train_input/std
test_input = test_input-mean
test_input = test_input/std


In [6]:
lr_reduction = 0
mseLoss = Project2_tools.MSE()
weight_bias = 0.5
epoch = 50
batch_size = 1
rounds = 5



In [11]:
def compute_error(input,target):
    nb_errors = 0
    for b in range(0, input.size(0)):
        output = model.forward(input[b])
        if ((output < 0 and target[b] == 1) or (output >= 0 and target[b] == -1)):
            nb_errors += 1 
    return nb_errors

In [12]:

lr = 0.005 # learning rate
train_results = []
test_results = []
for i in range(rounds):   
    print("round:",i)
    model = Project2_tools.Sequential(Project2_tools.Linear(2,25,weight_bias), Project2_tools.ReLU(),Project2_tools.Linear(25,25,weight_bias), Project2_tools.ReLU(),Project2_tools.Linear(25,25,weight_bias), Project2_tools.ReLU(),Project2_tools.Linear(25,1,weight_bias), Project2_tools.Tanh())

    for j in range(epoch):
        if lr_reduction and j%10 == 0:
            lr = lr*0.85
        loss = []
        for b in range(int(train_input.size(0)/batch_size)):
            for i in range(0, batch_size):
                output = model.forward(train_input[i+b*batch_size]) 
                loss.append(mseLoss.forward(output, train_target[i+b*batch_size])) 
                mseLoss_var = mseLoss.backward() 
                model.backward(mseLoss_var) 

            model.Stock_Grad_Descent(lr) 
            model.zero_grad() 
        loss_mean = torch.mean(torch.tensor(loss)) 


    nb_errors_train = compute_error(train_input,train_target)
    nb_errors_test = compute_error(test_input,test_target)
    
    
    print("Train Error : ",nb_errors_train/train_input.size(0)*100,"%")
    print("Test Error : ",nb_errors_test/test_input.size(0)*100,"%")
    print("")
    train_results.append(nb_errors_train/train_input.size(0)*100)
    test_results.append(nb_errors_test/test_input.size(0)*100)
    



round: 0
Train Error :  1.3 %
Test Error :  2.6 %

round: 1
Train Error :  1.6 %
Test Error :  3.0 %

round: 2
Train Error :  2.8000000000000003 %
Test Error :  3.2 %

round: 3
Train Error :  1.3 %
Test Error :  2.9000000000000004 %

round: 4
Train Error :  0.8999999999999999 %
Test Error :  2.7 %



In [13]:
train_mean =  sum(train_results)/len(train_results)
test_mean = sum(test_results)/len(test_results)

print("train mean:",train_mean)
print("test mean :",test_mean)

train mean: 1.58
test mean : 2.8800000000000003


In [14]:
train_results.sort()
test_results.sort()

print("train smalest:",train_results[0])
print("test smalest :",test_results[0])

train smalest: 0.8999999999999999
test smalest : 2.6
